In [1]:
import torch

import pandas as pd

from datetime import datetime

In [2]:
class IRT(torch.nn.Module):
    def __init__(self, df, q_col, u_col, s_col):
        super(IRT, self).__init__()
        
        self.df = df.sort_values([q_col, u_col])
        
        self.q = list(self.df[q_col].unique())
        self.q_to_index = {q : index for index, q in enumerate(self.q)}
        self.num_q = len(self.q)
        self.matrix_q = torch.nn.Parameter(torch.sparse_coo_tensor([list(range(len(self.df))), 
                                                                    [self.q_to_index[q] for q in self.df[q_col]]], 
                                                                   [1.] * len(self.df), 
                                                                   (len(self.df), self.num_q)).double(), 
                                           requires_grad=False)
        
        self.u = list(self.df[u_col].unique())
        self.u_to_index = {u : index for index, u in enumerate(self.u)}
        self.num_u = len(self.u)
        self.matrix_u = torch.nn.Parameter(torch.sparse_coo_tensor([list(range(len(self.df))), 
                                                                    [self.u_to_index[u] for u in self.df[u_col]]],
                                                                   [1.] * len(self.df), 
                                                                   (len(self.df), self.num_u)).double(), 
                                            requires_grad=False)
        
        self.s = torch.nn.Parameter(torch.tensor(self.df[s_col]).double(), requires_grad=False)
        
        self.theta = torch.nn.Parameter(torch.normal(torch.zeros(self.num_u), torch.ones(self.num_u)).double())
        self.a = torch.nn.Parameter(1 + (torch.rand(self.num_q).double() - 0.5) * 0.5)
        self.b = torch.nn.Parameter(torch.normal(torch.zeros(self.num_q), torch.ones(self.num_q)).double())
        self.c = torch.nn.Parameter(torch.rand(self.num_q).double() / 4)
        
    def forward(self):
        theta = torch.matmul(self.matrix_u, self.theta)
        a = torch.matmul(self.matrix_q, self.a)
        b = torch.matmul(self.matrix_q, self.b)
        c = torch.matmul(self.matrix_q, self.c)
        return c + (1 - c) / (1 + torch.exp(- a * (theta - b)))
    
    def predict(self, df, q_col, u_col):
        matrix_q = torch.sparse_coo_tensor([list(range(len(df))), 
                                            [self.q_to_index[q] for q in df[q_col]]], 
                                           [1.] * len(df), 
                                           (len(df), self.num_q)).double()
        matrix_u = torch.sparse_coo_tensor([list(range(len(df))), 
                                            [self.u_to_index[u] for u in df[u_col]]], 
                                           [1.] * len(df), 
                                           (len(df), self.num_u)).double()
        if self.s.is_cuda:
            matrix_q = matrix_q.cuda()
            matrix_u = matrix_u.cuda()
        
        theta = torch.matmul(matrix_u, self.theta)
        a = torch.matmul(matrix_q, self.a)
        b = torch.matmul(matrix_q, self.b)
        c = torch.matmul(matrix_q, self.c)
        return c + (1 - c) / (1 + torch.exp(- a * (theta - b)))

In [3]:
df = pd.read_csv('instance_questions_cs105.zip', compression='zip')
df = df[df.semester.isin(['fa22', 'sp23'])]
df = df[df.assessment_number.isin([1, 2, 3, 4])]
df = df.reset_index(drop=True).copy()

In [4]:
# objects are strings
df.dtypes

semester                           object
user_id                             int64
assessment_set_name                object
assessment_number                   int64
question_name                      object
question_type                      object
assessment_question_max_points      int64
instance_question_points          float64
instance_question_score_perc      float64
dtype: object

In [5]:
df.describe()

,user_id,assessment_number,assessment_question_max_points,instance_question_points,instance_question_score_perc
count,2.401000e+05,240100.000000,240100.000000,240100.000000,240100.000000
mean,4.177602e+06,2.537693,5.339396,4.356815,82.389007
std,6.894705e+05,1.154946,1.820471,2.413431,35.270243
min,1.299640e+05,1.000000,3.000000,0.000000,0.000000
25%,4.316444e+06,1.000000,3.000000,3.000000,100.000000
50%,4.319895e+06,3.000000,6.000000,5.000000,100.000000
75%,4.369411e+06,4.000000,6.000000,6.000000,100.000000
max,4.422763e+06,4.000000,9.000000,9.000000,100.000000


In [6]:
df['q'] = df['question_name']
df['u'] = df['semester'] + '_' + df['user_id'].astype(str) + '_' + df['assessment_set_name'] + '_' + df['assessment_number'].astype(str)
df['s'] = df['instance_question_score_perc'] / 100

In [7]:
irt = IRT(df, 'q', 'u', 's')
irt.cuda()
optimizer = torch.optim.AdamW(irt.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.999875)

In [8]:
for epoch in range(0, 20000):
    if epoch % 100 == 0:
        torch.save(irt, f'irt_models_l@s/irt_model_{epoch:05}.pkl')
    
    start = datetime.now()
    
    optimizer.zero_grad()
    
    s = irt().clamp(torch.finfo(torch.float64).eps, 1 - torch.finfo(torch.float64).eps)
    
    loss = sum(-irt.s * torch.log2(s) - (1 - irt.s) * (torch.log2(1 - s)))
    loss.backward()
    
    optimizer.step()
    scheduler.step()
    
    irt.theta.data.clamp_(-3, 3)
    irt.a.data.clamp_(0, 2)
    irt.b.data.clamp_(-3, 3)
    irt.c.data.clamp_(0, 1)
    
    end = datetime.now()
    print(f'epoch {epoch:4}, time {end - start}, loss {loss.item()}')

epoch    0, time 0:00:04.224609, loss 262390.8059924389
epoch    1, time 0:00:03.921849, loss 216252.184806718
epoch    2, time 0:00:04.030003, loss 188423.4645541006
epoch    3, time 0:00:03.997882, loss 170548.20145467945
epoch    4, time 0:00:03.934334, loss 158818.92010131353
epoch    5, time 0:00:03.934417, loss 151197.264890687
epoch    6, time 0:00:03.936342, loss 146470.35471112508
epoch    7, time 0:00:04.083100, loss 143867.09046570322
epoch    8, time 0:00:03.987341, loss 142878.4348414129
epoch    9, time 0:00:04.006213, loss 144956.16725211113
epoch   10, time 0:00:04.135728, loss 146379.4229463561
epoch   11, time 0:00:03.969941, loss 151592.15580695219
epoch   12, time 0:00:04.187847, loss 152494.10653768465
epoch   13, time 0:00:04.133931, loss 151513.7104511381
epoch   14, time 0:00:04.202561, loss 150035.04914608976
epoch   15, time 0:00:04.087051, loss 148847.18011260265
epoch   16, time 0:00:04.205375, loss 147521.62053047054
epoch   17, time 0:00:04.131825, loss 14

epoch  145, time 0:00:03.877291, loss 112120.1719460623
epoch  146, time 0:00:03.784035, loss 112116.41930026494
epoch  147, time 0:00:03.863637, loss 112112.77228397744
epoch  148, time 0:00:03.906247, loss 112109.20243345687
epoch  149, time 0:00:03.843724, loss 112105.67226205581
epoch  150, time 0:00:03.804782, loss 112102.21178737388
epoch  151, time 0:00:03.982305, loss 112098.71544362337
epoch  152, time 0:00:03.809417, loss 112061.14501510857
epoch  153, time 0:00:03.847106, loss 112050.11840221075
epoch  154, time 0:00:03.824442, loss 112048.72410301046
epoch  155, time 0:00:03.921084, loss 112047.39949818401
epoch  156, time 0:00:03.821175, loss 112045.55361061255
epoch  157, time 0:00:03.842951, loss 112042.8679648732
epoch  158, time 0:00:03.783818, loss 112039.48986080085
epoch  159, time 0:00:03.918380, loss 112035.71583693512
epoch  160, time 0:00:03.801204, loss 112031.83268002536
epoch  161, time 0:00:03.847552, loss 112028.05259898344
epoch  162, time 0:00:03.796609, 

epoch  290, time 0:00:03.793957, loss 111812.77202618838
epoch  291, time 0:00:03.831942, loss 111812.37861975063
epoch  292, time 0:00:03.879467, loss 111811.07187303262
epoch  293, time 0:00:03.842189, loss 111809.25337822114
epoch  294, time 0:00:03.809527, loss 111808.82179514802
epoch  295, time 0:00:03.926088, loss 111806.88164020702
epoch  296, time 0:00:03.813637, loss 111805.91707179627
epoch  297, time 0:00:03.869092, loss 111804.72105362882
epoch  298, time 0:00:03.836894, loss 111803.10387845708
epoch  299, time 0:00:03.822535, loss 111802.31558341016
epoch  300, time 0:00:03.882320, loss 111800.62471664115
epoch  301, time 0:00:03.856332, loss 111799.70548045648
epoch  302, time 0:00:03.804220, loss 111798.305622409
epoch  303, time 0:00:04.164392, loss 111797.00961964298
epoch  304, time 0:00:06.335405, loss 111795.93686804474
epoch  305, time 0:00:05.653998, loss 111794.40843018844
epoch  306, time 0:00:03.892857, loss 111793.41746694889
epoch  307, time 0:00:03.845189, 

epoch  435, time 0:00:03.855342, loss 111699.18252527333
epoch  436, time 0:00:03.793570, loss 111699.00536179647
epoch  437, time 0:00:03.929542, loss 111698.81917176518
epoch  438, time 0:00:03.904005, loss 111698.6308430562
epoch  439, time 0:00:03.849333, loss 111698.45983067785
epoch  440, time 0:00:03.817620, loss 111698.28053836309
epoch  441, time 0:00:03.912267, loss 111698.18931092272
epoch  442, time 0:00:03.797882, loss 111697.92409479029
epoch  443, time 0:00:03.824156, loss 111697.76488685168
epoch  444, time 0:00:03.947535, loss 111697.56371218147
epoch  445, time 0:00:03.850476, loss 111697.38719082896
epoch  446, time 0:00:03.794855, loss 111697.20537723953
epoch  447, time 0:00:03.959773, loss 111697.04857534663
epoch  448, time 0:00:03.843899, loss 111696.86269554165
epoch  449, time 0:00:03.837573, loss 111696.69427098689
epoch  450, time 0:00:03.802817, loss 111696.52973084581
epoch  451, time 0:00:03.857439, loss 111696.36692415521
epoch  452, time 0:00:03.897516,

epoch  580, time 0:00:03.795789, loss 111678.67138247825
epoch  581, time 0:00:03.852905, loss 111678.39332836066
epoch  582, time 0:00:03.903680, loss 111678.10883130504
epoch  583, time 0:00:03.857208, loss 111677.96823523885
epoch  584, time 0:00:03.847454, loss 111677.80301883564
epoch  585, time 0:00:03.840893, loss 111677.7148302029
epoch  586, time 0:00:03.905363, loss 111677.52157225566
epoch  587, time 0:00:03.900198, loss 111677.16199124258
epoch  588, time 0:00:03.798061, loss 111676.90568535942
epoch  589, time 0:00:03.946677, loss 111676.74675125409
epoch  590, time 0:00:03.852264, loss 111676.58868602119
epoch  591, time 0:00:03.869987, loss 111676.30171318482
epoch  592, time 0:00:03.831846, loss 111676.01660099335
epoch  593, time 0:00:03.886829, loss 111675.84521929722
epoch  594, time 0:00:03.833275, loss 111675.603019801
epoch  595, time 0:00:03.842161, loss 111675.35719594336
epoch  596, time 0:00:03.894152, loss 111675.02596197037
epoch  597, time 0:00:03.865144, l

...

epoch 19374, time 0:00:03.879122, loss 111519.15564443504
epoch 19375, time 0:00:03.870166, loss 111519.15564751338
epoch 19376, time 0:00:04.047105, loss 111519.1556610219
epoch 19377, time 0:00:03.950924, loss 111519.15566991513
epoch 19378, time 0:00:03.847003, loss 111519.15569003847
epoch 19379, time 0:00:04.056766, loss 111519.15570486561
epoch 19380, time 0:00:03.897804, loss 111519.1557396372
epoch 19381, time 0:00:03.886650, loss 111519.15576379868
epoch 19382, time 0:00:03.955966, loss 111519.1558275829
epoch 19383, time 0:00:03.955689, loss 111519.15586924652
epoch 19384, time 0:00:03.854095, loss 111519.15598971146
epoch 19385, time 0:00:03.951730, loss 111519.15606300834
epoch 19386, time 0:00:03.881074, loss 111519.15628292713
epoch 19387, time 0:00:03.965991, loss 111519.15642357022
epoch 19388, time 0:00:03.960846, loss 111519.15683456557
epoch 19389, time 0:00:04.001689, loss 111519.15709941446
epoch 19390, time 0:00:03.869454, loss 111519.15787096237
epoch 19391, time

epoch 19516, time 0:00:03.847153, loss 111519.15587167503
epoch 19517, time 0:00:03.967191, loss 111519.15562095855
epoch 19518, time 0:00:03.846249, loss 111519.15560999756
epoch 19519, time 0:00:03.954765, loss 111519.15577020026
epoch 19520, time 0:00:03.963963, loss 111519.15578269768
epoch 19521, time 0:00:03.972275, loss 111519.15565328592
epoch 19522, time 0:00:03.993486, loss 111519.15571167757
epoch 19523, time 0:00:04.061160, loss 111519.15576475879
epoch 19524, time 0:00:03.880455, loss 111519.15575472135
epoch 19525, time 0:00:03.950588, loss 111519.15560375295
epoch 19526, time 0:00:03.987055, loss 111519.15561293908
epoch 19527, time 0:00:03.903275, loss 111519.15562148542
epoch 19528, time 0:00:03.984582, loss 111519.15557404132
epoch 19529, time 0:00:03.989165, loss 111519.1554986746
epoch 19530, time 0:00:03.911520, loss 111519.15551975997
epoch 19531, time 0:00:04.216118, loss 111519.15555239619
epoch 19532, time 0:00:03.929784, loss 111519.15552684506
epoch 19533, ti

epoch 19658, time 0:00:03.955000, loss 111519.15690846564
epoch 19659, time 0:00:03.879302, loss 111519.15726578883
epoch 19660, time 0:00:03.851339, loss 111519.15688351248
epoch 19661, time 0:00:03.931593, loss 111519.15657574692
epoch 19662, time 0:00:04.002812, loss 111519.15613478712
epoch 19663, time 0:00:03.998434, loss 111519.15591447121
epoch 19664, time 0:00:03.923265, loss 111519.15562127948
epoch 19665, time 0:00:03.983790, loss 111519.15541316828
epoch 19666, time 0:00:03.865988, loss 111519.15544124617
epoch 19667, time 0:00:03.892897, loss 111519.15552636322
epoch 19668, time 0:00:03.914319, loss 111519.15564372498
epoch 19669, time 0:00:03.899933, loss 111519.15566751479
epoch 19670, time 0:00:03.836272, loss 111519.15581795454
epoch 19671, time 0:00:03.863001, loss 111519.15582944376
epoch 19672, time 0:00:03.958895, loss 111519.15584392703
epoch 19673, time 0:00:03.972985, loss 111519.1557125208
epoch 19674, time 0:00:04.109535, loss 111519.15568608827
epoch 19675, ti

epoch 19800, time 0:00:03.873358, loss 111519.15562463725
epoch 19801, time 0:00:03.882770, loss 111519.15553806424
epoch 19802, time 0:00:03.842537, loss 111519.15543466958
epoch 19803, time 0:00:03.890484, loss 111519.15536579222
epoch 19804, time 0:00:04.042945, loss 111519.15530507882
epoch 19805, time 0:00:03.969313, loss 111519.1552590446
epoch 19806, time 0:00:03.975562, loss 111519.15526963967
epoch 19807, time 0:00:03.920168, loss 111519.15526422921
epoch 19808, time 0:00:03.830115, loss 111519.15533138969
epoch 19809, time 0:00:03.862261, loss 111519.1553455726
epoch 19810, time 0:00:04.132819, loss 111519.15542126901
epoch 19811, time 0:00:03.926086, loss 111519.15541914172
epoch 19812, time 0:00:03.839502, loss 111519.155467235
epoch 19813, time 0:00:03.869966, loss 111519.15543959562
epoch 19814, time 0:00:04.078238, loss 111519.15545920722
epoch 19815, time 0:00:03.909780, loss 111519.15542286616
epoch 19816, time 0:00:04.034276, loss 111519.1554328961
epoch 19817, time 0

epoch 19942, time 0:00:03.867875, loss 111519.15540858012
epoch 19943, time 0:00:03.843164, loss 111519.15527180194
epoch 19944, time 0:00:04.019417, loss 111519.15515367972
epoch 19945, time 0:00:03.922225, loss 111519.15504500242
epoch 19946, time 0:00:04.017657, loss 111519.15498840541
epoch 19947, time 0:00:03.890373, loss 111519.15499586741
epoch 19948, time 0:00:03.855534, loss 111519.1550449914
epoch 19949, time 0:00:03.982238, loss 111519.15508257019
epoch 19950, time 0:00:03.877808, loss 111519.15509629168
epoch 19951, time 0:00:03.901575, loss 111519.15511501081
epoch 19952, time 0:00:03.888590, loss 111519.15507145207
epoch 19953, time 0:00:04.119876, loss 111519.15503989728
epoch 19954, time 0:00:03.899420, loss 111519.15498566258
epoch 19955, time 0:00:03.987292, loss 111519.15495776745
epoch 19956, time 0:00:03.867413, loss 111519.15494090017
epoch 19957, time 0:00:03.981456, loss 111519.15494472341
epoch 19958, time 0:00:03.890397, loss 111519.15496033004
epoch 19959, ti

In [9]:
torch.save(irt, f'irt_models_l@s/irt_model_final.pkl')